$\newcommand{\To}{\Rightarrow}$

In [1]:
import os
os.chdir('..')

In [2]:
from kernel.type import TFun, NatType, BoolType
from kernel.term import Var, Nat, Lambda, Abs, Bound, plus, Eq
from kernel.proofterm import ProofTerm, refl
from logic.conv import Conv, fun_conv, arg_conv, arg1_conv, rewr_conv, try_conv, binop_conv, beta_conv
from logic import basic
from syntax.settings import settings
from util import name

basic.load_theory('nat')
settings.unicode = True

## Rewriting on abstraction

In this section, we cover some further material on conversions. We begin where we left off in the last section: rewriting inside a $\lambda$-term. The conversion `abs_conv` takes a conversion as input, and applies the conversion to the body of a $\lambda$-term. It can be implemented as follows:

In [3]:
class abs_conv(Conv):
    def __init__(self, cv):
        self.cv = cv
        
    def get_proof_term(self, t):
        var_names = [v.name for v in t.body.get_vars()]
        nm = name.get_variant_name(t.var_name, var_names)
        v = Var(nm, t.var_T)
        body = t.subst_bound(v)
        
        return self.cv.get_proof_term(body).abstraction(v)

In line 1 of `get_proof_term`, we find the list of names of variables already in `t.body` using the `get_vars()` function. Note this only finds names of free variables, not the bound variable. On line 2, we use `name.get_variant_name` to find a fresh name for replacing the bound variable. Here `t.var_name` is the "suggested" name of the bound variable. The new variable $v$ is constructed in line 3, where `t.var_T` is the type of the bound variable. Finally, the body with bound variable replaced by $v$ is constructed on line 4.

Let's test the conversion on some examples.

In [4]:
x = Var('x', NatType)
t = Lambda(x, x + 2)
print('t: ', t)
pt = abs_conv(rewr_conv('add_comm')).get_proof_term(t)
print('pt:', pt)

t:  λx::nat. x + 2
pt: ProofTerm(⊢ (λx::nat. x + 2) = (λx. 2 + x))


The pattern in lines 1-4 of `abs_conv` occurs commonly, so it is implemented in the `dest_abs` function. We show some examples here:

In [5]:
t = Lambda(x, x + 2)
v, body = t.dest_abs()
print('t:', t)
print('v:', v)
print('body:', body)

t: λx::nat. x + 2
v: x
body: x + 2


We show another example with a name conflict:

In [6]:
t = Abs('x', NatType, plus(NatType)(x, Bound(0)))
v, body = t.dest_abs()
print('t:', t)
print('v:', v)
print('body:', body)

t: λx1. x + x1
v: x1
body: x + x1


Note how a fresh name `x1` is used both for printing and for the variable constructed by `dest_abs`. We now use `dest_abs` to simplify the implementation of `abs_conv`:

In [7]:
class abs_conv(Conv):
    def __init__(self, cv):
        self.cv = cv
        
    def get_proof_term(self, t):
        v, body = t.dest_abs()
        return self.cv.get_proof_term(body).abstraction(v)

In [8]:
t = Lambda(x, x + 2)
print(abs_conv(rewr_conv('add_comm')).get_proof_term(t))

ProofTerm(⊢ (λx::nat. x + 2) = (λx. 2 + x))


## Applying conversions to proof term

So far, we describe the usual API for implementing and using conversions in functional programming. In holpy, a different API is available, based on modifying proof terms using `on_prop` and `on_rhs` methods of class `ProofTerm`. We demonstrate this API below.

The `on_prop` method takes a conversion as input, and rewrites the proposition of the proof term using that conversion. For example:

In [9]:
P = Var('P', TFun(NatType, BoolType))
n = Var('n', NatType)

pt = ProofTerm.assume(P(n + 2))
pt2 = pt.on_prop(arg_conv(rewr_conv('add_comm')))
print('pt: ', pt)
print('pt2:', pt2)

pt:  ProofTerm(P (n + 2) ⊢ P (n + 2))
pt2: ProofTerm(P (n + 2) ⊢ P (2 + n))


It is important to note that calling `on_prop` on a proof term `pt` does *not* modify `pt`. Proof terms are immutable objects. Instead, the function returns a new proof term which is the result.

The `on_rhs` method applies the conversion to the right side of an equality. This is especially useful for composing conversions.

In [10]:
x = Var('x', NatType)
y = Var('y', NatType)
z = Var('z', NatType)

pt = refl(x * (y + z))
pt2 = pt.on_rhs(rewr_conv('distrib_l'))
pt3 = pt2.on_rhs(arg1_conv(rewr_conv('mult_comm')))
pt4 = pt3.on_rhs(arg_conv(rewr_conv('mult_comm')))
print('pt: ', pt)
print('pt2:', pt2)
print('pt3:', pt3)
print('pt4:', pt4)

pt:  ProofTerm(⊢ x * (y + z) = x * (y + z))
pt2: ProofTerm(⊢ x * (y + z) = x * y + x * z)
pt3: ProofTerm(⊢ x * (y + z) = y * x + x * z)
pt4: ProofTerm(⊢ x * (y + z) = y * x + z * x)


Multiple rewrites on the right side appears commonly, so we allow it to be done more conveniently by providing the conversions in sequence to the `on_rhs` function. This can replace most of the uses of `then_conv` and `every_conv`.

In [11]:
pt = refl(x * (y + z))
pt2 = pt.on_rhs(rewr_conv('distrib_l'), arg1_conv(rewr_conv('mult_comm')), arg_conv(rewr_conv('mult_comm')))
print('pt: ', pt)
print('pt2:', pt2)

pt:  ProofTerm(⊢ x * (y + z) = x * (y + z))
pt2: ProofTerm(⊢ x * (y + z) = y * x + z * x)


Incidentally, the function `binop_conv` can be used when we want to apply the same conversion to the two sides of a binary operator:

In [12]:
print(pt.on_rhs(rewr_conv('distrib_l'), binop_conv(rewr_conv('mult_comm'))))

ProofTerm(⊢ x * (y + z) = y * x + z * x)


We now apply these ideas to re-implement the `swap_cv` function. Recall the purpose of this conversion is to rewrite an expression of the form $(a + b) + c$ to $(a + c) + b$.

In [13]:
class swap_cv(Conv):
    def get_proof_term(self, t):
        return refl(t).on_rhs(
            rewr_conv('add_assoc'),
            arg_conv(rewr_conv('add_comm')),
            rewr_conv('add_assoc', sym=True))

In [14]:
t = (x + y) + 2
print('t: ', t)
print('pt:', refl(t).on_rhs(swap_cv()))

t:  x + y + 2
pt: ProofTerm(⊢ x + y + 2 = x + 2 + y)


This style of invoking conversions results in a more procedural code, better suited to the Python language.

## `top_conv` revisited

In the previous section on conversions, we implemented `top_conv` in the functional style. In this part, we re-implement the conversion in a more procedural style, also including rewriting on abstractions.

In [15]:
class top_conv(Conv):
    def __init__(self, cv):
        self.cv = cv
        
    def get_proof_term(self, t):
        pt = refl(t).on_rhs(try_conv(self.cv))
        if pt.rhs.is_comb():
            return pt.on_rhs(fun_conv(self), arg_conv(self))
        elif pt.rhs.is_abs():
            return pt.on_rhs(abs_conv(self))
        else:
            return pt

The function first tries to call `cv` on the current term. Next, if the result is not atomic (is either a combination or an abstraction), it calls `cv` on the subterms. Let's test the function on some examples.

In [16]:
x = Var('x', NatType)
y = Var('y', NatType)
z = Var('z', NatType)

t = Lambda(x, y, x * (3 * y + z + 1))
pt = refl(t).on_rhs(top_conv(rewr_conv('distrib_l')))
print('t: ', t)
print('pt:', pt)

t:  λx. λy. x * (3 * y + z + 1)
pt: ProofTerm(⊢ (λx. λy. x * (3 * y + z + 1)) = (λx. λy. x * (3 * y) + x * z + x * 1))


Likewise the class `bottom_conv` can be implemented as follows:

In [17]:
class bottom_conv(Conv):
    def __init__(self, cv):
        self.cv = cv
        
    def get_proof_term(self, t):
        pt = refl(t)
        if t.is_comb():
            return pt.on_rhs(fun_conv(self), arg_conv(self), try_conv(self.cv))
        elif t.is_abs():
            return pt.on_rhs(abs_conv(self), try_conv(self.cv))
        else:
            return pt.on_rhs(try_conv(self.cv))

Let's test it on the $\beta$-reduction example from before:

In [18]:
x = Var('x', NatType)
y = Var('y', NatType)
t = Lambda(x, y, x + y)(Nat(2), Nat(3))
print(refl(t).on_rhs(bottom_conv(beta_conv())))

ProofTerm(⊢ (λx::nat. λy. x + y) 2 3 = 2 + 3)


Classes like `top_conv` and `bottom_conv` are all implemented in the `Conv` module, and can be used directly.